In [1]:
import pandas as pd
import numpy as np
import json
import csv

RuntimeError: implement_array_function method already has a docstring

In [2]:
#read file
filename = 'iot-tweets-2009-2016-complet-enrichie.tsv'
data = pd.read_csv(filename,sep='\t', na_values=['null','None','',], dtype=object)
print(data['in_reply_to_status_id'][2])

# Iterative Root Finder

## Check if all Reply_to_id are in the corpus

In [5]:
replies_id =list(data[data['in_reply_to_status_id'].notnull()]['in_reply_to_status_id'])

In [6]:
print(len(replies_id)) # number of replies
print(len(set(replies_id))) # unique values of tweet replies

121170
111474


In [8]:
replies_not_in_corpus = set(replies_id)-set(data['TweetID'])
len(replies_not_in_corpus)

60848

## Get Dicts for faster searching

In [40]:
tweet_user = dict() # tweet->user
tweet_replies = dict() #tweet -> replies[]
user_replies = dict() #user -> replies[]
tweet_replied = dict() #tweet -> reply_to
def createDicts():
    
    for index,row in data[['TweetID','in_reply_to_status_id', 'User_ID','in_reply_to_user_id']].iterrows():
        reply_to = row['in_reply_to_status_id']
        if pd.isnull(reply_to)  or reply_to=='' or reply_to=='null':
            tweet_replied[row['TweetID']]='-1'
        else:
            tweet_replied[row['TweetID']]=row['in_reply_to_status_id']
            tweet_replies[row['in_reply_to_status_id']] = tweet_replies.get(row['in_reply_to_status_id'],[]) +[row['TweetID']]  
            user_replies[row['in_reply_to_user_id']] = tweet_replies.get(row['in_reply_to_user_id'],[]) + [row['User_ID']] 

        tweet_user[row['TweetID']]=row['User_ID']

In [41]:
from datetime import datetime
startTime = datetime.now()
createDicts()
print(datetime.now() - startTime) # runtime

0:13:02.337375


In [42]:
len(tweet_user), len(tweet_replies), len(user_replies), len(tweet_replied)

(6705948, 111474, 47951, 6705948)

In [77]:
# complementing dicts
for index,row in data[data['in_reply_to_status_id'].isin(replies_id)].iterrows():
    tweet_user[row['in_reply_to_status_id']] = row['in_reply_to_user_id']
    tweet_replied[row['in_reply_to_status_id']] = '-1'

In [78]:
len(tweet_user), len(tweet_replies), len(user_replies), len(tweet_replied)

(6766796, 111474, 47951, 6766796)

# Algorithm

In [49]:
visited = set()
roots = set()

def isReply(tweet_id):
    r = tweet_replied.get(tweet_id,'-1')
    if r=='-1':
        return False
    return True

def findRoot(tweet_id):
    cur_tweet = tweet_id
    found = True
    
    while isReply(cur_tweet):
        if cur_tweet in visited:
            found = False
            break
        visited.add(cur_tweet)
        cur_tweet = tweet_replied[cur_tweet]
    visited.add(cur_tweet)
    if found:
        roots.add(cur_tweet)

In [50]:
for key,val in tweet_replied.items():
#     print(key)
    findRoot(key)

In [51]:
len(roots), len(visited)

(6645626, 6766796)

In [52]:
#create  file
filenamet = 'roots.csv'

f = open(filenamet,'w')
f.write('roots\n')
for r in roots:
    f.write(r+"\n")
f.close()

# Iterative Search

## Algorithm

In [79]:
# tweet_user = dict() # tweet->user
# tweet_replies = dict() #tweet -> replies[]
# user_replies = dict() #user -> replies[]
# tweet_replied = dict() #tweet -> reply_to
def getReplies(tweet):
    user_id = tweet_user.get(tweet)
    ureps = set( user_replies.get(user_id,[]) )
    treps = set( tweet_replies.get(tweet,[]) ) 
    return treps, ureps
    

In [80]:
def iterativeSearch(tweet_id):
    set_t = set()
    set_u = set()
    set_t_old = set()
    set_u_old = set()
    set_empty = set()
    
    set_t.add(tweet_id)
    set_u.add(tweet_user[tweet_id])
    while (set_u_old ^ set_u) != set_empty or (set_t_old ^ set_t) != set_empty:
        P = set()
        M = set()
        for tweet in set_t:
            id_tweets, id_users = getReplies(tweet)
            M = M.union(id_users)
            P = P.union(id_tweets)
        set_t_old = set_t.copy()
        set_u_old = set_u.copy()
        set_t = set_t.union(P)
        set_u = set_u.union(M)
    return set_t, set_u
    

In [81]:
tweets_c = list()
users_c = list()
from datetime import datetime
startTime = datetime.now()

for root in roots: # search conversation thread for each root node
    set_tweets, set_users = iterativeSearch(root)
    tweets_c.append(set_tweets)
    users_c.append(set_users)
    
print(datetime.now() - startTime) # runtime

0:01:07.546099


In [82]:
len(roots), len(tweets_c), len(users_c)

(6645626, 6645626, 6645626)

In [84]:
#create converation tweets ID file
filename2 = 'conversations_tweet.txt'
n = len(tweets_c)
f = open(filename2,'w')
f.write(str(n) + "\n")
for conversation in tweets_c:
    for tweet in conversation:
        f.write(tweet+" ")
    f.write("\n")
f.close()

In [87]:
#create conversations Users ID file
filename3 = 'conversations_users.txt'
n = len(tweets_c)
f = open(filename3,'w')
f.write(str(n) + "\n")
for conversation in users_c:
    for user in conversation:
        f.write(str(user)+" ")
    f.write("\n")
f.close()

In [88]:
users_c

[{'306097253', '546412890'},
 {'2217209142', '229201150'},
 {'221088789', '713066829288177664'},
 {'743388655557869569'},
 {'2213836394'},
 {'3710546533'},
 {'4078708153'},
 {'875990875'},
 {'90686192'},
 {'2561907674', '4674421753'},
 {'10906472'},
 {'1133539693'},
 {'16205293'},
 {'15616564', nan},
 {'1283191', '51813565'},
 {'3389908396'},
 {'867305011'},
 {'26202871'},
 {'3155516198', '326119227'},
 {'301820203'},
 {'230440098', '49530613'},
 {'11230502', '2344273616'},
 {'53898323', '54514584'},
 {'26582238', '72596449'},
 {'23442383', '3185325258'},
 {'2367320131', '3232185336'},
 {'1141258524', '3041470176'},
 {'14436232', '67008212'},
 {'15616564', nan},
 {'250987479', '5784402'},
 {'17178964', '6249802'},
 {'15616564', nan},
 {'15616564', nan},
 {'4096844362'},
 {'4899357672'},
 {'2860630333', '3908968426'},
 {'15616626', '19406718'},
 {'15616564', nan},
 {'1536148147'},
 {'1090315298', '1923637494'},
 {'2697859279', '4674421753'},
 {'15616564', nan},
 {'3318013381', '61203130

In [89]:
tweets_c

[{'579371454585823233'},
 {'745419447649763328'},
 {'630046376366374912'},
 {'795723594345103361'},
 {'669002236735787008'},
 {'655940961038561280'},
 {'768855602395217920'},
 {'603569459018641410'},
 {'459528396234850305'},
 {'780753791641645056'},
 {'789607962901749760'},
 {'477079928996110336'},
 {'542600379599310848'},
 {'768256987092123652'},
 {'551672789590343681'},
 {'794253279010160641'},
 {'610231514098475008'},
 {'447596959214608384'},
 {'475199894815539200'},
 {'508837331176927232'},
 {'776012787227254784'},
 {'472395796361801729'},
 {'797051588972056576'},
 {'631232403378298881'},
 {'603152464859234304'},
 {'807605269593210880'},
 {'618853215145209856'},
 {'525324616752177152'},
 {'742744950903873536'},
 {'686293971333582850'},
 {'411544221602963456'},
 {'485453445231419395'},
 {'683813176631275520'},
 {'775284523445747713'},
 {'718499892730003457'},
 {'753095751719391233'},
 {'697894694215225344'},
 {'805875799542861828'},
 {'693502822248235008'},
 {'425989737837785088'},


In [98]:
data[data['TweetID']=='735032838009950208']

,TweetID,Sentiment,TopicID,Country,Gender,URLs,Text,User_ID,User_Name,Date,Hashtags,in_reply_to_status_id,Indication,Retweeted,Geo,Location,in_reply_to_user_id
4914596,735032838009950208,neutral,2,fr,andy,https://t.co/72tdHP8XIQ,'Welcome is on TV! Follow us &amp; RT to win y...,478502804,'netatmo','Mon Jan 30 10:46:09 +0000 2012',"[{'text': 'followRT', 'indices': [92, 101]}, {...",NaN,tweet,false,NaN,'',NaN


In [91]:
data[data['User_ID']=='306097253']

,TweetID,Sentiment,TopicID,Country,Gender,URLs,Text,User_ID,User_Name,Date,Hashtags,in_reply_to_status_id,Indication,Retweeted,Geo,Location,in_reply_to_user_id
2284912,601831303877500928,neutral,NaN,in,andy,https://twitter.com/BernardMarr/status/6018299...,'Internet of Things is Immortal and Eternal.. ...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'IoT', 'indices': [118, 122]}, {'tex...",NaN,tweet,false,NaN,'',NaN
2296229,602673393134014464,neutral,5,in,andy,https://twitter.com/WSJ/status/602670934168842240,'Audi Relocates to The #InternetOfThings ........,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'InternetOfThings', 'indices': [22, ...",NaN,tweet,false,NaN,'',NaN
2357569,605251486159765504,neutral,3,in,andy,https://twitter.com/_jcbales/status/6052504132...,'French Forward Finally \n\n#BigData\n#IoT\n#L...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'BigData', 'indices': [0, 8]}, {'tex...",NaN,tweet,false,NaN,'',NaN
2357640,605255490923134976,neutral,3,in,andy,https://twitter.com/kanter/status/604329162883...,'Curating #Cyberchondria \n\n#Leadership \n#Mo...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',[],NaN,tweet,false,NaN,'',NaN
2357695,605258162015322112,neutral,NaN,in,andy,https://twitter.com/EconomicTimes/status/60525...,"""Flipkarts Daring Duo's Self-Drive\n\n#Softwar...",306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'software', 'indices': [17, 26]}, {'...",NaN,tweet,false,NaN,'',NaN
2357815,605264097714696192,neutral,3,in,andy,https://rjmetrics.com/resources/reports/ecomme...,'Ecommerce Buyer Behavior https://t.co/QSs6TX0...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'Leadership', 'indices': [50, 61]}, ...",NaN,tweet,false,NaN,'',NaN
2357822,605264591585574913,neutral,3,in,andy,https://psmag.com/economics/the-high-cost-of-c...,'High Cost of Cynicism http://t.co/97eFphgybD\...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'Leadership', 'indices': [46, 57]}, ...",NaN,tweet,false,NaN,'',NaN
2357829,605265286363648000,neutral,2,in,andy,https://www.washingtonpost.com/business/capita...,'#IoT Roadmap.D Internet is about 2 change aga...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'IoT', 'indices': [0, 4]}, {'text': ...",NaN,tweet,false,NaN,'',NaN
2357861,605267187763273728,neutral,3,in,andy,https://twitter.com/SEETEK_AU/status/605254969...,'Measure of Success ...\n\n#Leadership \n#Moti...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',"[{'text': 'quote', 'indices': [16, 22]}, {'tex...",NaN,tweet,false,NaN,'',NaN
2357975,605270817207762944,neutral,NaN,in,andy,https://twitter.com/indddian/status/6052652878...,'To each his Own ..\n\n#Leadership \n#Motivati...,306097253,'DrSanjayPSahoo','Fri May 27 08:38:26 +0000 2011',[],NaN,tweet,false,NaN,'',NaN
